## Fine tuning the convolutional layers

The code below shows how we can unfreeze last few layers to allow fine tuning

In [1]:
from __future__ import print_function

import os
import json
import shutil
import numpy as np

from utils import prepare_data

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, roc_curve, \
                            precision_recall_curve, average_precision_score, confusion_matrix
import pickle
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

## Prepare Data

In [2]:
data_path = "data"
models_path = "models"
valid_size = 0.2
FORCED_DATA_REWRITE = False

In [3]:
train_path, valid_path = prepare_data(data_path=data_path, 
                                      valid_size=valid_size, 
                                      FORCED_DATA_REWRITE=FORCED_DATA_REWRITE)

In [4]:
train_neg_path = os.path.join(train_path, "Negative")
train_pos_path = os.path.join(train_path, "Positive")
valid_neg_path = os.path.join(valid_path, "Negative")
valid_pos_path = os.path.join(valid_path, "Positive")

In [6]:
img_height, img_width = 150, 150

In [7]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [8]:
train_gen = datagen.flow_from_directory(train_path, 
                                        target_size=(img_height, img_width), 
                                        class_mode='binary', 
                                        batch_size=16, 
                                        shuffle=False)

valid_gen = datagen.flow_from_directory(valid_path, 
                                        target_size=(img_height, img_width), 
                                        class_mode='binary', 
                                        batch_size=16, 
                                        shuffle=False)

Found 1305 images belonging to 2 classes.
Found 325 images belonging to 2 classes.


## Pre-trained Model as Feature Extractor

In [9]:
conv_base = VGG19(include_top=False, 
                         weights="imagenet",  
                         input_shape=(img_height, img_width, 3))

conv_base.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [10]:
model = Sequential()
model.add(conv_base)
model.add(GlobalAveragePooling2D())
model.add(Dense(units=512, activation="relu", kernel_initializer="he_normal"))
model.add(Dropout(rate=0.5))
model.add(Dense(units=1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 4, 4, 512)         20024384  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 20,287,553
Trainable params: 20,287,553
Non-trainable params: 0
_________________________________________________________________


In [15]:
conv_base.trainable = False
# only unfreeze the last conv block (block5_convx) 
for layer in conv_base.layers:
    if layer.name in ['block5_conv4']:
        layer.trainable = True


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 4, 4, 512)         20024384  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 20,287,553
Trainable params: 263,169
Non-trainable params: 20,024,384
_________________________________________________________________


In [17]:
model.compile(loss="binary_crossentropy", 
                  optimizer=Adam(lr=0.0001), 
                  metrics=["accuracy"])

In [18]:
train_steps_per_epoch = int(np.ceil(train_gen.n * 1. / train_gen.batch_size))
valid_steps_per_epoch = int(np.ceil(valid_gen.n * 1. / valid_gen.batch_size))

In [19]:
history = model.fit_generator(
      train_gen,
      steps_per_epoch=train_steps_per_epoch,
      epochs=20,
      validation_data=valid_gen, 
      validation_steps=valid_steps_per_epoch,
      verbose=2)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
82/82 - 10s - loss: 0.7870 - accuracy: 0.4958 - val_loss: 0.6870 - val_accuracy: 0.5385
Epoch 2/20
82/82 - 8s - loss: 0.7706 - accuracy: 0.4904 - val_loss: 0.6800 - val_accuracy: 0.5662
Epoch 3/20
82/82 - 9s - loss: 0.7444 - accuracy: 0.5272 - val_loss: 0.6567 - val_accuracy: 0.6154
Epoch 4/20
82/82 - 9s - loss: 0.7170 - accuracy: 0.5632 - val_loss: 0.6642 - val_accuracy: 0.5908
Epoch 5/20
82/82 - 9s - loss: 0.7115 - accuracy: 0.5617 - val_loss: 0.6512 - val_accuracy: 0.6338
Epoch 6/20
82/82 - 9s - loss: 0.6919 - accuracy: 0.5724 - val_loss: 0.6386 - val_accuracy: 0.6431
Epoch 7/20
82/82 - 9s - loss: 0.6997 - accuracy: 0.5678 - val_loss: 0.6247 - val_accuracy: 0.6431
Epoch 8/20
82/82 - 9s - loss: 0.6829 - accuracy: 0.5931 - val_loss: 0.6516 - val_accuracy: 0.6185
Epoch 9/20
82/82 - 9s - loss: 0.6644 - accuracy: 0.5870 - val_loss: 0.6194 - val_accuracy: 0.6431
Epoch 10/20
82/82 - 9s - loss: 0.6593 - a

The validation accuracy seems to be worse than before (i.e. without fine-tuning the convolutional layer and just trained the classification dense layer). One reason maybe that our training samples are too little. 